In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from datetime import datetime

# Load Dataset
df = pd.read_csv("merged_file.csv", encoding="latin1")

# Convert timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Select features for anomaly detection
features = [
    "Z-Axis RMS Velocity (in/sec)",
    "Temperature (°F)",
    "Z-Axis Peak Acceleration (G)",
    "X-Axis Peak Acceleration (G)",
    "Z-Axis Peak Velocity (mm/sec)",
    "X-Axis RMS Velocity (in/sec)",
]
X = df[features]

# Train Isolation Forest model
model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
model.fit(X)
df['Anomaly'] = model.predict(X)
df['Anomaly'] = df['Anomaly'].map({1: 0, -1: 1})  # Convert to 0 (normal) and 1 (anomaly)

# Define anomaly detection function for live data
def detect_anomaly(live_data):
    live_df = pd.DataFrame([live_data], columns=features)
    prediction = model.predict(live_df)
    return "Anomaly" if prediction[0] == -1 else "Normal"

# Example real-time monitoring simulation
live_data_example = {
    "Z-Axis RMS Velocity (in/sec)": 0.2,
    "X-Axis RMS Velocity (in/sec)": 0.1,
    "Temperature (°F)": 92,
    "Z-Axis Peak Acceleration (G)": 0.8,
    "X-Axis Peak Acceleration (G)": 0.05,
    "Z-Axis Peak Velocity (mm/sec)": 5,
    "X-Axis Peak Velocity (mm/sec)": 5
}

print("Live Data Anomaly Detection:", detect_anomaly(live_data_example))

# Save the trained model
import joblib
joblib.dump(model, "isolation_forest_model.pkl")

C:\Users\Harsh\AppData\Local\Temp\ipykernel_16820\2606095903.py:10: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Timestamp'] = pd.to_datetime(df['Timestamp'])


KeyError: "['Temperature (°F)'] not in index"

In [2]:
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import IsolationForest
# from datetime import datetime
# import joblib

# # Load Dataset
# df = pd.read_csv("vibration_data.csv", encoding="latin1")

# # Convert timestamp to datetime
# df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# # Select features for anomaly detection
# features = [
#     "Z-Axis RMS Velocity (in/sec)", "X-Axis RMS Velocity (in/sec)",
#     "Temperature (°F)", "Z-Axis Peak Acceleration (G)", "X-Axis Peak Acceleration (G)",
#     "Z-Axis Peak Velocity (mm/sec)", "X-Axis Peak Velocity (mm/sec)"
# ]
# X = df[features]

# # Load the trained Isolation Forest model
# model = joblib.load("isolation_forest_model.pkl")

# df['Anomaly'] = model.predict(X)
# df['Anomaly'] = df['Anomaly'].map({1: 0, -1: 1})  # Convert to 0 (normal) and 1 (anomaly)

# # Define anomaly detection function for live data
# def detect_anomaly(live_data):
#     live_df = pd.DataFrame([live_data], columns=features)
#     prediction = model.predict(live_df)
#     return "Anomaly" if prediction[0] == -1 else "Normal"

# # Example real-time monitoring simulation
# live_data_example = {
#     "Z-Axis RMS Velocity (in/sec)": 0.2,
#     "X-Axis RMS Velocity (in/sec)": 0.1,
#     "Temperature (°F)": 92,
#     "Z-Axis Peak Acceleration (G)": 0.8,
#     "X-Axis Peak Acceleration (G)": 0.05,
#     "Z-Axis Peak Velocity (mm/sec)": 5,
#     "X-Axis Peak Velocity (mm/sec)": 5
# }

# print("Live Data Anomaly Detection:", detect_anomaly(live_data_example))

# # Save the dataset with anomaly labels
# df.to_csv("vibration_data_with_anomalies.csv", index=False)


In [3]:
# pip install minimalmodbus 

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from datetime import datetime
import joblib
import time
from pymodbus.client import ModbusSerialClient

# Load the trained Isolation Forest model
model = joblib.load("isolation_forest_model.pkl")

# Modbus RTU Configuration
MODBUS_PORT = "COM7"  # Change based on your system
BAUDRATE = 19200
STOPBITS = 1
BYTESIZE = 8
PARITY = "N"
TIMEOUT = 3

# Initialize Modbus RTU Client
client = ModbusSerialClient(
    port=MODBUS_PORT,
    baudrate=BAUDRATE,
    stopbits=STOPBITS,
    bytesize=BYTESIZE,
    parity=PARITY,
    timeout=TIMEOUT
)

# Register Addresses (Subtract 1 for pymodbus)
REGISTER_MAPPING = {
    "Z-Axis RMS Velocity (in/sec)": 45201 - 40001,
    "X-Axis RMS Velocity (in/sec)": 45205 - 40001,
    "Temperature (°F)": 45203 - 40001,
    "Z-Axis Peak Acceleration (G)": 45207 - 40001,
    "X-Axis Peak Acceleration (G)": 45208 - 40001,
    "Z-Axis Peak Velocity (mm/sec)": 45218 - 40001,
    "X-Axis Peak Velocity (mm/sec)": 45220 - 40001,
}

# Scaling Factors
SCALING_FACTORS = {
    "Z-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "X-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "Temperature (°F)": 327.67 / 32767,
    "Z-Axis Peak Acceleration (G)": 65.535 / 65535,
    "X-Axis Peak Acceleration (G)": 65.535 / 65535,
    "Z-Axis Peak Velocity (mm/sec)": 65.535 / 65535,
    "X-Axis Peak Velocity (mm/sec)": 65.535 / 65535,
}

def read_modbus_data():
    """ Reads sensor data from Modbus RTU """
    if not client.connect():
        print("❌ Failed to connect to Modbus RTU device")
        return None
    
    live_data = {}
    for feature, address in REGISTER_MAPPING.items():
        try:
            response = client.read_holding_registers(address=address, count=1, slave=1)
            if response and response.registers:
                raw_value = response.registers[0]
                live_data[feature] = round(raw_value * SCALING_FACTORS[feature], 3)
            else:
                live_data[feature] = None
        except Exception as e:
            print(f"Error reading {feature}: {e}")
            live_data[feature] = None
    return live_data

def detect_anomaly(live_data):
    if not live_data:
        return "Error"
    live_df = pd.DataFrame([live_data], columns=REGISTER_MAPPING.keys())
    prediction = model.predict(live_df)
    return "Anomaly" if prediction[0] == -1 else "Normal"

def monitor_live_data():
    print("✅ Monitoring live data from Modbus RTU...")
    while True:
        live_data = read_modbus_data()
        if not live_data or None in live_data.values():
            print("⚠️ Error reading Modbus data, skipping this cycle.")
            time.sleep(2)
            continue
        
        result = detect_anomaly(live_data)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if result == "Anomaly":
            print(f"[{timestamp}] 🚨 ALERT! Anomaly detected: {live_data}")
        else:
            print(f"[{timestamp}] ✅ Normal data: {live_data}")
        
        time.sleep(2)  # Monitoring interval

# Start live monitoring
monitor_live_data()

client.close()  # Ensure Modbus client is closed before re-running


✅ Monitoring live data from Modbus RTU...
[2025-02-11 10:50:41] ✅ Normal data: {'Z-Axis RMS Velocity (in/sec)': 0.021, 'X-Axis RMS Velocity (in/sec)': 0.034, 'Temperature (°F)': 89.94, 'Z-Axis Peak Acceleration (G)': 0.038, 'X-Axis Peak Acceleration (G)': 0.038, 'Z-Axis Peak Velocity (mm/sec)': 0.741, 'X-Axis Peak Velocity (mm/sec)': 1.214}
[2025-02-11 10:50:43] ✅ Normal data: {'Z-Axis RMS Velocity (in/sec)': 0.021, 'X-Axis RMS Velocity (in/sec)': 0.034, 'Temperature (°F)': 89.94, 'Z-Axis Peak Acceleration (G)': 0.038, 'X-Axis Peak Acceleration (G)': 0.038, 'Z-Axis Peak Velocity (mm/sec)': 0.741, 'X-Axis Peak Velocity (mm/sec)': 1.214}
[2025-02-11 10:50:45] ✅ Normal data: {'Z-Axis RMS Velocity (in/sec)': 0.022, 'X-Axis RMS Velocity (in/sec)': 0.031, 'Temperature (°F)': 89.94, 'Z-Axis Peak Acceleration (G)': 0.038, 'X-Axis Peak Acceleration (G)': 0.042, 'Z-Axis Peak Velocity (mm/sec)': 0.8, 'X-Axis Peak Velocity (mm/sec)': 1.114}
[2025-02-11 10:50:47] ✅ Normal data: {'Z-Axis RMS Velocit

KeyboardInterrupt: 